In [1]:
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.autograd import Variable
import pandas as pd
from torch.utils.data import Dataset,DataLoader


In [2]:
class diabete_dataset(Dataset):
    
    def __init__(self):
        df=pd.read_csv('diabetes.csv',header=None,sep=',',dtype=np.float32)
        self.x_data=Variable(torch.from_numpy(df[[0, 1, 2, 3, 4, 5, 6, 7]].as_matrix()))
        self.y_data=Variable(torch.from_numpy(df[[8]].as_matrix()))
        self.len=df.shape[0]
                      
    def __getitem__(self,index):
        
        return self.x_data[index],self.y_data[index]
    
    def __len__(self):
                       
        return self.len

In [3]:
dataset_=diabete_dataset()

In [4]:
train_loader=DataLoader(dataset=dataset_,batch_size=64,shuffle=True)

In [5]:
data,label=next(iter(train_loader))

In [6]:
data.size()

torch.Size([64, 8])

In [7]:
class Network(torch.nn.Module):
    
    def __init__(self):
        super(Network,self).__init__()
        self.fc1=torch.nn.Linear(8,6)
#        self.dropout1=torch.nn.Dropout(p=0.2)
        self.fc2=torch.nn.Linear(6,4)
        self.dropout2=torch.nn.Dropout(p=0.2)
        self.fc3=torch.nn.Linear(4,2)
        self.fc4=torch.nn.Linear(2,1)
        #self.sigmoid=torch.nn.Sigmoid()
        
        
    def forward(self,x):
        
        out1=F.relu(self.fc1(x))
#        out1=self.dropout1(out1)
        out2=F.relu(self.fc2(out1))
        out2=self.dropout2(out2)
        out3=F.relu(self.fc3(out2))
        out4=F.sigmoid(self.fc4(out3))   #sof max at end, or sigmoid all the way did not work out
        y_pred=out4
        return y_pred
        

In [8]:
model=Network()

In [9]:
criterion=torch.nn.BCELoss(size_average=True)  #if increase the lr to 0.1, it kicks to local min and stays
optimizer=torch.optim.Adam(model.parameters(),lr=0.01)  #had to change from SGD to Adam but later noy much difference
lr_scheduler=torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')


In [10]:
torch.set_printoptions(precision=2)
epochs=1500
loss_running=0
count=0
count_batch=0
sum_acc=0

for epoch in range(epochs):

    for input,label in iter(train_loader):
        
        y_pred=model(input)
        loss=criterion(y_pred,label)
        loss_running=loss_running+loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        count=count+1
        y_pred_round=torch.round(y_pred)
        count_batch=count_batch+(label.size()[0])
  
    
        acc=(label==y_pred_round).sum().item()    #/label.size()[0]
        sum_acc=sum_acc+acc
    check_loss=(loss_running/count_batch)
    lr_scheduler.step(check_loss)
    if(epoch%100==0): print('epoch',epoch,'/',epochs,'running_loss:',(loss_running/count_batch),'Accuracy',(sum_acc*100/count_batch))
    
    

epoch 0 / 1500 running_loss: tensor(1.00000e-02 *
       1.13) Accuracy 34.65085638998683
epoch 100 / 1500 running_loss: tensor(1.00000e-03 *
       7.79) Accuracy 74.64485578992682
epoch 200 / 1500 running_loss: tensor(1.00000e-03 *
       7.31) Accuracy 76.43141342038162
epoch 300 / 1500 running_loss: tensor(1.00000e-03 *
       7.10) Accuracy 77.20466254338852
epoch 400 / 1500 running_loss: tensor(1.00000e-03 *
       6.97) Accuracy 77.66157728209122
epoch 500 / 1500 running_loss: tensor(1.00000e-03 *
       6.89) Accuracy 77.9218900801822
epoch 600 / 1500 running_loss: tensor(1.00000e-03 *
       6.82) Accuracy 78.19466457967506
epoch 700 / 1500 running_loss: tensor(1.00000e-03 *
       6.77) Accuracy 78.37702209717344
epoch 800 / 1500 running_loss: tensor(1.00000e-03 *
       6.73) Accuracy 78.54740204520371
epoch 900 / 1500 running_loss: tensor(1.00000e-03 *
       6.69) Accuracy 78.6632917019444
epoch 1000 / 1500 running_loss: tensor(1.00000e-03 *
       6.66) Accuracy 78.757606